Draw mask from VIA - .csv file

In [1]:
#Importação das bibliotecas necessárias para rodar o código
%matplotlib inline

import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
import re

In [2]:
#Define o mapa de cores para geração da máscara
def colorir(cat):
    h, w = cat.shape[:2]
    msk = np.zeros((h,w,3), dtype = 'uint8')
    msk[cat == 1] = [255,255,255]
    msk[cat == 2] = [128,0,0]
    msk[cat == 3] = [255,0,0]
    msk[cat == 4] = [0,128,0]
    msk[cat == 5] = [0,255,0]
    msk[cat == 6] = [0,0,255]
    return(msk)

In [3]:
def get_mask(imgt):
    #Split Image in 3 channels
    blue,green,red = cv2.split(imgt)
    #Normalize values
    green = np.divide(green,255)
    red = np.divide(red,255)
    blue = np.divide(blue,255)

    #Sum of normalized values
    norm = red+green+blue

    r = np.divide(red, norm, out=np.zeros_like(red), where=norm!=0)
    g = np.divide(green, norm, out=np.zeros_like(green), where=norm!=0)
    b = np.divide(blue, norm, out=np.zeros_like(blue), where=norm!=0)
    
    #ExG - ExR
    midx = ((2*g) - r - b) - ((1.4*r) - g)

    #Binarize image
    midx[midx<0] = 0
    midx[midx>0] = 1
    imager = (midx*255).astype('uint8')
    return(imager)

In [4]:
#Definição das pastas necessárias para rodar o script
img_dir = 'C:/Users/mhmce/SmartAgri/Curso_Deep_Learning/Corn/'
pts_dir = 'C:/Users/mhmce/SmartAgri/Curso_Deep_Learning/pts/'
pts_col_dir = 'C:/Users/mhmce/SmartAgri/Curso_Deep_Learning/pts_col/'
anotacao = 'C:/Users/mhmce/SmartAgri/Curso_Deep_Learning/via_region_data_FS.csv'

In [5]:
#Apresenta as imagens dentro do diretório com o respectivo formato da imagem
img_list = os.listdir(img_dir)
img_list = [v for v in img_list if v.endswith('.JPG')]
print(img_list)

['DJI_0392.JPG', 'DJI_0393.JPG', 'DJI_0394.JPG', 'DJI_0420.JPG']


In [25]:
csv_file = pd.read_csv(anotacao,index_col = False) #Faz a leitura do arquivo .csv

for img in img_list:
    lista = csv_file[csv_file.filename == img] #Recebe os dados encontrados dentro do csv para a imagem em questão
    tamanho = lista.shape
    imagem = cv2.imread(img_dir+img) #Carrega a imagem em questão
    msk = np.zeros(imagem.shape[:2], dtype = 'uint8') #Carrega as informações do tamanho da imagem
    # Cria uma mascara auxiliar para o desenho dos circulos ao redor das plantas:
    msk0 = msk.copy()            
    # Importa a mascara do indice de vegetacao:
    mskk = get_mask(imagem)
    mskk = (mskk > 100).astype('uint8') 
    msk_name = os.path.join(pts_dir, img.replace('.JPG', '.png')) #Faz a junção do diretório com o nome da imagem, alterando seu formato

    
    if tamanho[0] != 0:
        for i in range(tamanho[0]):
            line = lista.iloc[i,:] #Recebe todas as marcações realizadas dentro daquela imagem
            region_shape = line.region_shape_attributes #Informa a posição onde o ponto se encontra
            region_attributes = (line.region_attributes) #Informa a classe do ponto
            region_attributes = re.findall('"([^"]*)"', region_attributes) #Faz a separação para encontrar a qual classe pertence o ponto marcado
            coords = [float(s) for s in re.findall(r'-?\d+\.?\d*', region_shape)] #Encontrando valores de x e y
            cx = int(coords[0]) #Coordenadas no eixo X
            cy = int(coords[1]) #Coordenadas no eixo y

            #Define as classes aqui para realizar o desenho da máscara      
            if region_attributes[1] == 'Corn':
                cv2.circle(msk0, (cx,cy), 50, 2, -1)
                
            if region_attributes[1] == 'Corn':
                cv2.circle(msk0, (cx,cy), 10, 6, -1)
        
        #Faz a combinação das mascaras na máscara principal
        msk[(msk0==2)&(mskk==1)] = 2
        msk[(msk0==6)&(mskk==1)] = 6
                
        print(msk_name) #Apresenta o nome da máscara com o diretório a ser salvo
        cv2.imwrite(msk_name, msk) #Realiza o salvamento do background
    
        col_msk = colorir(msk) #Realiza a coloração da máscara
        col_msk_name = os.path.join(pts_col_dir, img.replace('.JPG', '.png')) #Faz a junção do diretório com o nome da imagem, alterando seu formato
        cv2.imwrite(col_msk_name, col_msk) #Realiza o salvamento da máscara já colorida
        
    if tamanho[0] == 0:
        print('A imagem'+' '+img+' '+'não possui anotação no .csv apresentado') #Apresenta uma mensagem caso a imagem lida não apresente informações dentro do .csv

C:/Users/mhmce/SmartAgri/Curso_Deep_Learning/pts/DJI_0392.png
C:/Users/mhmce/SmartAgri/Curso_Deep_Learning/pts/DJI_0393.png
C:/Users/mhmce/SmartAgri/Curso_Deep_Learning/pts/DJI_0394.png
A imagem DJI_0420.JPG não possui anotação no .csv apresentado
